In [38]:
import pandas as pd
df = pd.read_csv('/home/tbdbj/forest_test/marina_results_derev/qween_ds_1.csv', delimiter=',')
df

,file_name,categories,texts
0,e6f22245ae1048c98e292c829b9ef12a.jpg,"05.07 QR-код / адрес сайта,24.03 Лекарственные...","Реклама лекарственных средств, Реклама медици..."
1,baebf04a69874393837cccb862ee9dd0.jpg,"24.03 Лекарственные препараты,24.06 МедОрганиз...","Реклама лекарственных средств, Реклама медици..."
2,bfbd14b99e4c4a24aafe908c4bf5cd6f.jpg,"28.07 Строительство (ДДУ),28.08 Застройщик","Реклама недвижимости, Реклама услуг по строит..."
3,71252cd88c014113b44d2d911e1aee55.jpg,"01.01 Прочая реклама,05.03 Информационная прод...","Реклама телевизионного сериала, Реклама телев..."
4,96257d02e86446b1817b255f9554e0e1.jpg,"01.01 Прочая реклама,05.03 Информационная прод...","Реклама телевизионного канала, Реклама телеви..."
...,...,...,...
17629,6ea60afc76e24cd2b3b95608e2427628.png,10.01 Социальная реклама,"Реклама товаров, работ, услуг, Реклама событи..."
17630,eabd1fa824df4e30b28bf312495ceded.jpg,"28.07 Строительство(ДДУ),28.08 Застройщик,28.1...","Реклама недвижимости, Реклама ипотеки, Реклам..."
17631,fa32ca9aca5745f58943580d19ffa20e.jpg,02.04 Объявление о продаже,"Телекоммуникации, Техника для дома, Электроника"
17632,a8fdedf775ab4c48a7980720ebdcbf05.jpg,"05.03 Информационная продукция,08.01 Дистанцио...","Реклама культурных мероприятий, Реклама спект..."


In [71]:
from typing import List
import re

class ReTextFinder:
  def __init__(self, words_list: List[List[str]]):
    self.__patterns = []
    shift = 0
    for words in words_list:
      dict = {}
      for i, word in enumerate(words):
        dict[word] = shift + i
      shift += len(words)
      self.__patterns.append(
        (
          dict,
          re.compile(
            "(%s)" % "|".join(
              map(re.escape, sorted([w+' ' for w in words], key=lambda s: len(s), reverse=True))
            )
          )
        )
      )

  def finditer(self, text:str):
    hash = set()
    text += ' '
    for dict, patterns in self.__patterns:
      for mo in patterns.finditer(text):
        index = dict[mo.group()[:-1]]
        if index in hash:
          continue
        hash.add(index)
        yield index
  
  @property
  def features(self)->List[str]:
    return [k for d, _ in self.__patterns  for k, _ in d.items()]

In [91]:
textfinder = ReTextFinder(
                [["Ипотека"], ['в ипотеку'], ['ипотека предоставляется', "кредит"], ['в кредит'], ['кредит предоставляется'], 
               [ "жилищный кредит","первоначальный взнос","процентная ставка"],["господдержка"], ['с господдержкой', 
                'военная ипотека'], ["ипотека для"], ["субсидированная ипотека", "Срок кредита", "застройщик берёт",  "на себя платежи"], 
                ['займы выдаются', 'без открытия банковского'],['в выдаче займа', 'договора залога', 'платежи по ипотеке']])
for i in textfinder.finditer("ипотека предоставляется первоначальный взнос с господдержкой кредит"):
  print(i)

2
3
7
10


In [90]:
textfinder = ReTextFinder(
                [["Ипотека"], ['в ипотеку'], ['ипотека предоставляется', "кредит"], ['в кредит'], ['кредит предоставляется'], 
               [ "жилищный кредит","первоначальный взнос","процентная ставка"],["господдержка"], ['с господдержкой', 
                'военная ипотека'], ["ипотека для"], ["субсидированная ипотека", "Срок кредита", "застройщик берёт",  "на себя платежи"], 
                ['займы выдаются', 'без открытия банковского'],['в выдаче займа', 'договора залога', 'платежи по ипотеке']])
for i in textfinder.finditer("процентная ставка первоначальный взнос субсидированная ипотека без открытия банковского"):
  print(i)

8
7
13
18


In [81]:
textfinder.features

['Ипотека',
 'в ипотеку',
 'ипотека предоставляется',
 'кредит',
 'в кредит',
 'кредит предоставляется',
 'жилищный кредит',
 'первоначальный взнос',
 'процентная ставка',
 'господдержка',
 'с господдержкой',
 'военная ипотека',
 'ипотека для',
 'субсидированная ипотека',
 'Срок кредита',
 'застройщик берёт',
 'на себя платежи',
 'займы выдаются',
 'без открытия банковского',
 'в выдаче займа',
 'договора залога',
 'платежи по ипотеке']